# Sistema de Gestión de Eventos con MCP Remoto (HTTP)

## Descripción

En este ejercicio, construirás un **agente con un servidor MCP remoto auto-hospedado**, que expone herramientas HTTP reutilizables para gestionar reservas de salas y consultar eventos programados en un centro corporativo.

### Arquitectura MCP Remoto
```
Pregunta sobre eventos o salas
    ↓
[EventAgent]
    ↓
[MCPStreamableHTTPTool] → http://<codespaces-url>/api/mcp
    ↓
Servidor Python (remote_mcp_server.py)
    ↓
[Herramientas del servidor]
    ├→ get_available_rooms()
    └→ get_upcoming_events()
    ↓
Análisis y respuesta
```

### 2. Ejecutar el Servidor en GitHub Codespaces

**Pasos:**
1. Abre una nueva terminal en GitHub Codespaces (`Terminal > New Terminal`).
2. Ejecuta el servidor con el siguiente comando: `python remote_mcp_server.py`
3. Codespaces detectará automáticamente que el puerto `8000` está siendo utilizado y te mostrará una notificación para hacer el reenvío de puertos (port forwarding).
4. Haz clic en **Open in Browser** o copia la URL proporcionada para acceder al servidor MCP.
5. En el panel **Ports**, cambia manualmente la visibilidad del puerto reenviado de **Private** a **Public**.
6. Pega la URL pública resultante + `/api/mcp` en la variable `CODESPACES_FORWARDED_URL` de la celda de código más abajo.

### 3. Configurar la URL expuesta por Codespaces

Define la URL pública reenviada para reutilizarla en las consultas del agente.

In [ ]:
# Replace this value with your forwarded URL ending with /api/mcp
CODESPACES_FORWARDED_URL = "https://super-duper-happiness-gjwppqrvwj72wqr-8000.app.github.dev/api/mcp"

### 4. Cargar librerías y variables de entorno para el cliente

In [ ]:
import asyncio
import os
from dotenv import load_dotenv, find_dotenv
from agent_framework import ChatAgent, MCPStreamableHTTPTool
from agent_framework.openai import OpenAIChatClient

load_dotenv(find_dotenv(usecwd=True))

# Configure the GitHub Models client
MODEL_NAME = os.getenv("GITHUB_MODEL", "openai/gpt-4o")

### 5. Crear y ejecutar el agente cliente

In [ ]:
async def query_events(prompt, server_url):
    if not server_url or "YOUR_URL_HERE" in server_url:
        print("Error: Please set CODESPACES_FORWARDED_URL with the public server URL.")
        return

    print(f"Connecting to the remote MCP server at {server_url}...")
    
    try:
        client = OpenAIChatClient(
            model_id=MODEL_NAME,
            api_key=os.environ["GITHUB_TOKEN"],
            base_url="https://models.github.ai/inference"
        )
        
        async with (
            MCPStreamableHTTPTool(
                name="eventmanagementservice",
                url=server_url,
            ) as mcp_server,
            ChatAgent(
                chat_client=client,
                name="EventAgent",
                instructions=(
                    "Eres un asistente de gestión de eventos que puede consultar salas disponibles y eventos programados a través de un servidor remoto."
                ),
            ) as agent,
        ):
            print("Connected to the remote MCP server.")
            result = await agent.run(prompt, tools=mcp_server)
            print("Result:")
            print(result)
    except Exception as e:
        print(f"Error while connecting to the MCP server: {type(e).__name__}")
        print(f"Details: {str(e)}")

### 6. Probar el agente con diferentes consultas

A continuación, probaremos el agente con distintos tipos de consultas para verificar su capacidad de usar las herramientas del servidor MCP remoto.

In [ ]:
# Consulta para forzar el uso de la herramienta get_available_rooms
user_prompt = "Muestra todas las salas disponibles con su capacidad y equipamiento."
await query_events(user_prompt, CODESPACES_FORWARDED_URL)

In [ ]:
# Consulta para forzar el uso de la herramienta get_upcoming_events
user_prompt = "Muestra todos los eventos programados para la próxima semana."
await query_events(user_prompt, CODESPACES_FORWARDED_URL)

In [ ]:
# Prompt complejo: asignar salas para nuevos eventos verificando disponibilidad y capacidad
user_prompt = """
Escenario: El departamento de Marketing quiere organizar 3 eventos más esta semana:
- Evento A: 45 personas, necesita proyector y sistema de audio
- Evento B: 15 personas, necesita pizarra digital
- Evento C: 100 personas, necesita escenario y sonido profesional

Basándote en las salas disponibles y los eventos ya programados:
1. Asigna una sala apropiada para cada nuevo evento
2. Verifica que no haya conflictos de capacidad
3. Si alguna sala propuesta ya tiene un evento, sugiérelo explícitamente
4. Proporciona alternativas si la primera opción no está disponible
"""
await query_events(user_prompt, CODESPACES_FORWARDED_URL)

In [ ]:
# Prompt complejo: calcular ocupación de salas y optimizar recursos
user_prompt = """
Analiza los eventos programados y las salas reservadas. Identifica:
1. Qué salas están ocupadas y cuándo
2. Qué salas NO tienen eventos asignados
3. Calcula el porcentaje de ocupación de salas
4. Detecta si hay algún evento que podría reubicarse a una sala más pequeña para optimizar recursos
Proporciona un análisis detallado con números y recomendaciones.
"""
await query_events(user_prompt, CODESPACES_FORWARDED_URL)